# Learning interpretable models for performance analysis and tuning

[Pooyan Jamshidi](https://github.com/pooyanjamshidi),
[Christian Kaestner](https://www.cs.cmu.edu/~ckaestne/)

In this tutorial, we will present how to learn black-box models that capture the influence of configuration options and their interactions on performance of configurable systems. Performance influence models [1] are human interpretable models that enable udnerestanding what options are more important than others (i.e., affect performance more significantly), and how options interact (i.e., if they are enabled together they influence the performance differently if selected individually).

```latex
[1] N. Siegmund, A. Grebhahn, S. Apel, and C. Kästner. Performance-influence models for highly con gurable systems. In Proc. Software Engineering Conf. Foundations of Software Engineering (ESEC/FSE), ACM, August 2015.
```

# Installations

First, you need to install `python3`:
* MacOS: http://docs.python-guide.org/en/latest/starting/install3/osx/
* Linux: http://docs.python-guide.org/en/latest/starting/install3/linux/
* Windows: https://www.python.org/downloads/windows/

Then, you need to make sure you installed `pip`:
```bash
python3 -m pip install pip
```
Assuming you installed `git`, you then do:
```bash
git clone https://github.com/cmu-mars/model-learner.git
cd model-learner
git checkout tutorial
python3 -m pip install --upgrade .
```
Now, you are ready to start!

# Import libraries

In [1]:
# import the libraries, classes, methods we need for this tutorial
from learner.mlearner import learn_with_interactions, learn_without_interactions, mean_absolute_percentage_error, sample_random, stepwise_feature_selection
from learner.model import genModelTermsfromString, Model, genModelfromCoeff
import numpy as np

def evalModel(model, config):
    return float(model.evaluateModelFast(np.matrix(config))[0])
def learnModel(X, y, withInteractions):
    if (withInteractions):
        m = learn_with_interactions(X, y)
    else:
        m = learn_without_interactions(np.asarray(X), np.asarray(y))
    learned_model_terms = genModelfromCoeff(m.named_steps['linear'].coef_, ndim)
    return Model(learned_model_terms, ndim)

# for this tutorial we assume 20 options:
ndim = 20 # defines the dimension of the model, i.e., the number of variables in the model


# Manually defining configurations

## Configurations:

In [14]:
# lets see how we evaluate the model with specific input
c0 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c1 = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c2 = [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c3 = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c23 = [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

## Measurements:

In [15]:
#Let's define a list of configurations and a corresponding list of measurement results
X = [c0, c1, c2, c3, c23]
y = [10, 12, 15, 13, 10]

X, y


([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [10, 12, 15, 13, 10])

## Learn the model without interaction terms

In [16]:
learned_model = learnModel(X, y, 0)
learned_model.toString()

'-0.00 * o0 + 1.00 * o1 + -1.00 * o2 + -6.043984975198235e-16'

## Learn the model with pairwise interaction terms

In [5]:
learned_model = learnModel(X, y, 1)
learned_model.toString()

'2.00 * o0 + 5.00 * o1 + 3.00 * o2 + -8.00 * o0 * o19 + -8.00 * o1 * o18 + -8.00 * o2 * o17 + -8.00 * o3 * o16 + -8.00 * o4 * o15 + -8.00 * o5 * o14 + -8.00 * o6 * o13 + -8.00 * o7 * o12 + -8.00 * o8 * o11 + -8.00 * o9 * o10 + -2.6645352591003757e-15'

# Using a secret model as ground truth

In [6]:
# Let's define a model, in this case a polynomial model
# Polynomial models are a great tool for determining which input factors drive responses and in what direction.
# Here we define a model with 20 dimensions, each variable represents a dimension and influence of each variable
# is different, e.g., o0 has the coefficient of 1, while o1 has the coefficient of 2 so its effect is twice comparing
# with o0. Also, we have two terms that represents the interactions of variables, e.g., 3 * o3 * o6.

true_model = """10 + 1.00 * o0 + 2.00 * o1 + 3.00 * o2 +
4.00 * o3 + 5.00 * o4 + 6.00 * o5 + 7.00 * o6 + 8.00 * o7 + 
1.00 * o8 + 2.00 * o9 + 3.00 * o10 + 4.00 * o11 + 5.00 * o12 + 
6.00 * o13 + 7.00 * o14 + 8.00 * o15 + 1.00 * o16 + 2.00 * o17 + 
3.00 * o18 + 4.00 * o19 + 1 * o0 * o1 + 3 * o3 * o6"""

In [7]:
# The model above is just a representation in string, so we need to build a model that we can evaluate given an input
model_terms = genModelTermsfromString(true_model)
true_model = Model(model_terms, ndim)
true_model.toString()

'1.00 * o0 + 2.00 * o1 + 3.00 * o2 + 4.00 * o3 + 5.00 * o4 + 6.00 * o5 + 7.00 * o6 + 8.00 * o7 + 1.00 * o8 + 2.00 * o9 + 3.00 * o10 + 4.00 * o11 + 5.00 * o12 + 6.00 * o13 + 7.00 * o14 + 8.00 * o15 + 1.00 * o16 + 2.00 * o17 + 3.00 * o18 + 4.00 * o19 + 1.00 * o0 * o1 + 3.00 * o3 * o6 + 10.0'

## Sampling from the secret model

In [8]:
# We could also look up specific measurement results in a secret model
# that we use as ground truth:
# sample_random(ndim=ndim, budget=1000)

y = true_model.evaluateModelFast(np.asarray(X))

X, y

([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 array([ 10.,  11.,  12.,  13.,  15.]))

In [9]:
# instead of manually picking configurations, we can also let the system pick random ones:

# budget = 10000
# X = sample_random(ndim=ndim, budget=budget)
# y = true_model.evaluateModelFast(X)
# X, y

## Learn the model without interaction terms

In [10]:
learned_model = learnModel(X, y, 0)
learned_model.toString()

'1.00 * o0 + 2.00 * o1 + 3.00 * o2'

## Learn the model with pairwise interaction terms

In [11]:
learned_model = learnModel(X, y, 1)
learned_model.toString()

'1.00 * o0 + 2.00 * o1 + 3.00 * o2 + 0.00 * o0 * o19 + 0.00 * o1 * o18 + 0.00 * o2 * o17 + 0.00 * o3 * o16 + 0.00 * o4 * o15 + 0.00 * o5 * o14 + 0.00 * o6 * o13 + 0.00 * o7 * o12 + 0.00 * o8 * o11 + 0.00 * o9 * o10 + -4.440892098500626e-16'

## Evaluate the learning

In [31]:
# we first consider sampling some ground truth data, note that we sample again 
# in order to test on data that was not necessarily used for learning the model
X = sample_random(ndim=ndim, budget=100)
y_true = true_model.evaluateModelFast(X)
y_pred = learned_model.evaluateModelFast(X)
err = mean_absolute_percentage_error(y_true=y_true, y_pred=y_pred)
err

100.03169798554327

In [32]:
stepwise_feature_selection(X, y_true)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/learner/mlearner.py:87: FutureWarning: 'argmin' is deprecated. Use 'idxmin' instead. The behavior of 'argmin' will be corrected to return the positional minimum in the future. Use 'series.values.argmin' to get the position of the minimum now.
  best_feature = new_pval.argmin()


Add o7                             with p-value 1.06313e-06
Add o14                            with p-value 2.65496e-05
Add o15                            with p-value 1.08871e-06
Add o6                             with p-value 3.88792e-08
Add o13                            with p-value 4.63556e-10
Add o4                             with p-value 1.03303e-12
Add o5                             with p-value 5.07077e-15
Add o9                             with p-value 2.35972e-16
Add o1                             with p-value 1.30455e-17
Add o12                            with p-value 9.45739e-19
Add o3                             with p-value 9.07295e-20
Add o18                            with p-value 1.35352e-20
Add o0                             with p-value 1.73368e-21
Add o19                            with p-value 1.46169e-22
Drop o7                             with p-value 0.0636851
Add o7                             with p-value 1.90586e-19
Drop o6                             with 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/learner/mlearner.py:100: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  worst_feature = pvalues.argmax()


Add o6                             with p-value 1.90586e-19
Drop o5                             with p-value 0.0636851
Add o5                             with p-value 1.90586e-19
Drop o4                             with p-value 0.0636851
Add o4                             with p-value 1.90586e-19
Drop o3                             with p-value 0.0636851
Add o11                            with p-value 4.76751e-21
Add o3                             with p-value 3.80866e-26
Add o10                            with p-value 1.19178e-32
Add o2                             with p-value 2.5724e-38
Add o17                            with p-value 7.53838e-47
Add o16                            with p-value 6.65819e-53
Add o8                             with p-value 3.09175e-59


[14, 15, 13, 9, 1, 12, 18, 0, 19, 7, 6, 5, 4, 11, 3, 10, 2, 17, 16, 8]